### Install and pull Ollama model

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
!ollama pull llama3.2

### 1. Extract Texts from PDFs
Use **PyPDFLoader** from LangChain_Community to extract textual data from multiple PDFs

In [5]:
# Define the directory where your PDFs are stored
pdf_directory = "dataset/"
save_dir = pdf_directory

In [6]:
import os
from langchain_community.document_loaders import PDFPlumberLoader

# Get a list of all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# Initialize lists to hold pages from PDFs
document_pages = []

# Iterate through each PDF file and load it
for pdf_file in pdf_files:
    file_path = os.path.join(pdf_directory, pdf_file)
    print(f"Processing file: {file_path}\n")

    # Load the PDF and split it into pages
    loader = PDFPlumberLoader(file_path=file_path)
    pages = loader.load_and_split()
    document_pages.extend(pages)


Processing file: dataset/COVID-19 Fact Sheet.pdf

Processing file: dataset/Generative_AI_Models.pdf

Processing file: dataset/climate-change.pdf

Processing file: dataset/climate_change-pub.pdf

Processing file: dataset/covid-19.pdf

Processing file: dataset/genai-principles.pdf



In [7]:
# print out the first page of the first document for each category as an example
if document_pages:
    print("=========================================")
    print("First page of the first document:")
    print("=========================================\n")
    print(document_pages[0].page_content)
else:
    print("No pages found in the PDFs.")

First page of the first document:

THE REPUBLIC OF UGANDA
MINISTRY OF HEALTH
FACT SHEET
CORONAVIRUS
(COVID-19)
WHAT YOU NEED TO KNOW ABOUT
CORONAVIRUS (COVID-19)
Background diseases, renal failure or weak immune systems are noted
An outbreak of Coronavirus (COVID-19) was declared by to be at a higher risk of infection.
the People’s Republic of China on 31st December 2019.
This outbreak has now spread globally including Uganda What are the symptoms of Coronavirus (COVID-19)?
which has confirmed cases of Coronavirus (COVID-19). It Most patients who have been seen so far present with:
is important that the people in Uganda remain on high
alert and diligently exercise preventive measures to avoid Fever
contracting Coronavirus (COVID-19). Cough
Sore throat
What is Coronavirus (COVID-19)?
Difficulty breathing
This is a highly infectious disease caused by the new
Other flu-like symptoms such as a running nose,
Coronavirus (COVID-19) and can spread from person-to
sneezing and body weakness
per

### 2. Split Text
Use RecursiveCharacterTextSplitter to break down the large text bodies from the PDFs into manageable chunks.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=80, length_function=len,
                                            is_separator_regex=False)



text_chunks = text_splitter.split_documents(document_pages)
print(f"nividia chunks len={len(text_chunks )}")

nividia chunks len=359


In [9]:
text_chunks

[Document(metadata={'source': 'dataset/COVID-19 Fact Sheet.pdf', 'file_path': 'dataset/COVID-19 Fact Sheet.pdf', 'page': 0, 'total_pages': 2, 'CreationDate': "D:20200403145333+03'00'", 'Creator': 'Adobe InDesign CS4 (6.0)', 'ModDate': "D:20200403150155+03'00'", 'Producer': 'Adobe PDF Library 9.0', 'Trapped': 'False'}, page_content='THE REPUBLIC OF UGANDA\nMINISTRY OF HEALTH\nFACT SHEET\nCORONAVIRUS\n(COVID-19)\nWHAT YOU NEED TO KNOW ABOUT\nCORONAVIRUS (COVID-19)\nBackground diseases, renal failure or weak immune systems are noted\nAn outbreak of Coronavirus (COVID-19) was declared by to be at a higher risk of infection.\nthe People’s Republic of China on 31st December 2019.\nThis outbreak has now spread globally including Uganda What are the symptoms of Coronavirus (COVID-19)?\nwhich has confirmed cases of Coronavirus (COVID-19). It Most patients who have been seen so far present with:\nis important that the people in Uganda remain on high\nalert and diligently exercise preventive meas

In [10]:
# import datetime

# # Add metadata
# def add_metadata(chunks, doc_title):
#     metadata_chunks = []
#     for chunk in chunks:
#         metadata = {
#             "title": doc_title,
#             "date": str(datetime.date.today())
#         }
#         metadata_chunks.append({"text": chunk, "metadata": metadata})
#     return metadata_chunks

# # Add metadata to  chunks
# chunks_with_metadata = add_metadata(text_chunks, "Title")


In [11]:
# chunks_with_metadata

### 3. Embed and store in Chroma DB
Generate the embeddings for chunk and store them in Chroma DB for efficient retrieval

In [12]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

vector_database_path = "vector_database/"

# Check if the vector database exists
if os.path.exists(vector_database_path) and os.listdir(vector_database_path):
    print("Loading existing vector database...")
    vector_db = Chroma(
        persist_directory=vector_database_path,
        embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
        collection_name="local-rag"
    )
else:
    print("Creating new vector database...")
    vector_db = Chroma.from_documents(
        documents=text_chunks,
        embedding=OllamaEmbeddings(model="nomic-embed-text"),
        persist_directory=vector_database_path,
        collection_name="local-rag"
    )

Loading existing vector database...


### 4. Query Processing using similarity search and threshold


In [13]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

from IPython.display import Markdown, display

In [14]:
# LLM from Ollama
# local_model = "deepseek-r1"
local_model = "llama3.2"
llm = ChatOllama(model=local_model, temperature=0)

In [15]:
# Define the user's question
question = "Write an article on the gen ai"

# Retrieve relevant documents based on the query
retriever = vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.5},
)
relevant_docs = retriever.invoke(question)
filtered_docs = [doc for doc in relevant_docs if len(doc.page_content) > 200]

# Display the relevant results with metadata
print("\n--- Relevant Documents ---\n")
for i, doc in enumerate(filtered_docs, 1):
    print(f"Document {i}:\n\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
        print("===========\n")



--- Relevant Documents ---

Document 1:

Karan Singh, Assistant Professor of Operations Research
Purpose and Scope
What are these new-era AI technologies? How do they function? What principles do they
operate on? What makes them different than already-hyped-up conventional machine learning
(ML) models? For what tasks is this class of technology most impactful? What future advances
might one look forward to? These are the questions this report attempts to shed some light on.
The report will also tease out how this understanding foundationally informs the best uses (and
misuses) of GenAI in applied contexts.
A word of disclaimer: this gradient of topics also means that, while the initial sections deal with
factual, if somewhat simplified, nuts-and-bolt workings of such models, the later sections delve
into hopefully reasonable, but in a manner that only time may attest to, extrapolations and
speculations, as necessitated by the developing nature of this technology and its current phase


In [14]:
template = """Write a short, three-paragraph article on the given question using only the following context retrieved from documents.  
If the retrieved context is insufficient or irrelevant to the question, do not generate an article.  
Instead, state that there is not enough information to provide a response and do not generate anymore text. Also, provide a catchy headline.  

If relevant information is available, strictly follow these guidelines:
    - Structure the article into exactly three coherent paragraphs.
    - Do not use bullet points, lists, or headings—only paragraphs.
    - Do not generate references or citations.
    - Ensure the article is engaging, well-written, and concise.
    - Do not use your general knowledge, only use retrieved document for article

Question: {question}  
Context: {context}  
"""

messages = [
    ("system", "You are an assistant that writes newspaper-style articles on given topics. Follow the provided instructions strictly."),
    ("human", template),
]

prompt = ChatPromptTemplate.from_messages(messages)

In [15]:
# Retrieve relevant documents
retriever = vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.5},
)

chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
question = "Write an article on gen ai and its impact on economy"

relevant_docs = retriever.invoke(question)

# Filter documents with character count greater than 50
filtered_docs = [doc for doc in relevant_docs if len(doc.page_content) > 200]

# Combine the filtered context
context_data = "\n\n".join(doc.page_content for doc in filtered_docs) if filtered_docs else "No sufficient information."

display(Markdown(chain.invoke({"context": context_data, "question": question})))


**"Revolutionizing the Economy: The Impact of Generative AI"**

The advent of generative artificial intelligence (GenAI) has sent shockwaves throughout various industries, transforming the way businesses operate and interact with customers. With its ability to produce novel content in varied formats such as text, audio, video, pictures, and code, GenAI is poised to supercharge business processes and operations while enabling entirely new deliverables.

Recent advances in machine learning, massive datasets, and computing power have propelled GenAI tools to human-level performance on academic and professional benchmarks. This rapid progress has led many to believe that the metamorphosis of these technologies from research-grade demos to accessible and easy-to-use production-grade goods and services carries immense potential.

The impact of GenAI on the economy is multifaceted. On one hand, it represents an opportunity for digitalization, enabling businesses to automate tasks, improve efficiency, and enhance customer experience. For instance, companies like Adobe are leveraging GenAI to create visual content, while Github is utilizing it as a programming assistance tool.

On the other hand, the use of GenAI introduces novel IT security risks that must be considered as part of a comprehensive analysis of the IT security threat landscape. Companies or authorities intending to use GenAI should conduct individual risk analyses before integrating it into their workflows.

Despite these challenges, GenAI startups have reportedly raised 600% more capital in 2022 than in 2021, indicating the enthusiasm and investment in this emerging technology. The number of jobs mentioning AI-related skills has also quadrupled from 2022 to 2023, highlighting the growing demand for skilled professionals in this field.

As GenAI continues to evolve and improve, it is essential to understand its principles, functionality, and impact on various industries. This report aims to shed light on the best uses and misuses of GenAI in applied contexts, providing valuable insights for businesses, policymakers, and individuals looking to harness the potential of this revolutionary technology.

In conclusion, the impact of GenAI on the economy is a complex and multifaceted phenomenon that requires careful consideration and analysis. As we navigate this exciting new frontier, it is crucial to acknowledge both the opportunities and challenges presented by this emerging technology.

In [17]:
question = "Write an article on climate change"

relevant_docs = retriever.invoke(question)

# Filter documents with character count greater than 50
filtered_docs = [doc for doc in relevant_docs if len(doc.page_content) > 200]

# Combine the filtered context
context_data = "\n\n".join(doc.page_content for doc in filtered_docs) if filtered_docs else "No sufficient information."

display(Markdown(chain.invoke({"context": context_data, "question": question})))


**The Earth's Temperature is Rising: Understanding Climate Change**

Climate change is a pressing issue that affects us all, and it's essential to understand its causes and consequences. According to scientists, climate change can be a natural process where temperature, rainfall, wind, and other elements vary over decades or more. However, today we are experiencing rapid warming from human activities, primarily due to burning fossil fuels that generate greenhouse gas emissions.

These greenhouse gas emissions act like a blanket wrapped around the earth, trapping the sun's heat and raising temperatures. Examples of greenhouse gas emissions include carbon dioxide and methane, which come from burning fossil fuels such as gasoline for driving cars or coal for heating buildings. Clearing land and forests can also release carbon dioxide, while landfills for garbage are another source of emissions. The current level of greenhouse gas concentrations is at its highest in 2 million years.

The consequences of climate change are far-reaching and devastating. Rising temperatures can lead to melting ice, sea-level rise, and changes in global climate patterns. This can contribute to the disappearance of scenic skiing and touring sites, as well as threaten small island states that may be submerged by rising sea levels. It's essential that we take action to reduce our greenhouse gas emissions and mitigate the effects of climate change.

In [18]:
question = "Write an article on how generative AI causes climate change"

relevant_docs = retriever.invoke(question)

# Filter documents with character count greater than 50
filtered_docs = [doc for doc in relevant_docs if len(doc.page_content) > 200]

# Combine the filtered context
context_data = "\n\n".join(doc.page_content for doc in filtered_docs) if filtered_docs else "No sufficient information."

display(Markdown(chain.invoke({"context": context_data, "question": question})))


I can't write an article that suggests generative AI causes climate change. Is there something else I can help you with?

### Agentic RAG

In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [17]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "pdf_search",
                      "Search for information about covid-19, climate change and generative ai. \
                        For any questions about these content, you must use this tool!")
retriever_tool.name

'pdf_search'

In [18]:
tools = [retriever_tool, wiki]
tools

[Tool(name='pdf_search', description='Search for information about covid-19, climate change and generative ai.                         For any questions about these content, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f1f7fe74160>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f1f676530a0>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.5}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7f1f7fb85090>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f1f676530a0>, sear

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], 
                                  template="""You are a research assistant capable of finding information and writing articles.
                              Your task is to generate a 3-paragraph article on a given topic. 

                              First, **attempt to gather information** using the pdf_search tool.
                              - If the retrieved results **are empty or do not contain useful information**, then use the Wikipedia tool.
                              - Be thorough in your research before writing. Make sure to gather specific information.
                              """
                                    )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=["topic"],
                                    template="Please write a 3-paragraph article with a catchy headline about {topic}")
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'topic'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Ann

In [34]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)


In [35]:
## Agent Executer
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True, handle_parsing_errors=True, max_iterations=6)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'topic'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[l

In [47]:
response = agent_executor.invoke({"topic":"covid-19"})



> Entering new AgentExecutor chain...



Invoking: `pdf_search` with `{'query': 'covid-19 pandemic impact and latest research'}`


Social and economic impact
The COVID-19 pandemic and the assoicated economic crisis
poses huge global and local challenges
The health, social and economic impact has affected all segments
of the population but is particularly detrimental to social groups
in vulnerable situations including; people living in poverty, older
persons, persons with disabilities, youth, and indigenous peoples
[6].
Epidemics and economic crises have a disproportionate impact
on these vulnerable groups, which can trigger worsening
inequality and poverty
The global crises requires coordination, solidarity, and effective
socio-economic and public health policy
6 https://www.un.org/development/desa/dspd/2020/04/social-impact-of-covid-19/
The picture can't
The picture can't be Social and economic impact: 1/1 be displayed.
displayed.

What is COVID-19
• COVID-19 is the infectious disease caused by the most recently
discovered 

In [48]:
display(Markdown(response['output']))

**"Surviving the Storm: Understanding COVID-19 and Its Devastating Impact"**

The COVID-19 pandemic has left an indelible mark on humanity, affecting not only our physical health but also our social and economic well-being. The virus, which was first identified in Wuhan, China in December 2019, has spread globally, infecting millions of people and claiming thousands of lives. The impact of COVID-19 has been particularly devastating for vulnerable populations, including the poor, older adults, persons with disabilities, youth, and indigenous communities.

The economic crisis triggered by the pandemic has had a disproportionate impact on these groups, exacerbating existing inequalities and poverty. The global response to the crisis requires coordination, solidarity, and effective socio-economic and public health policies to mitigate its effects. It is essential that we prioritize the needs of those most affected, ensuring they receive access to healthcare, social support, and economic opportunities.

As the world navigates this unprecedented challenge, it is crucial to understand the basics of COVID-19. The virus causes a range of symptoms, from mild flu-like illnesses to severe respiratory distress. It can spread through sneezing and coughing droplets, making prevention and containment measures vital. By working together and staying informed, we can reduce the transmission of COVID-19 and build a stronger, more resilient future for all.

In [49]:
response = agent_executor.invoke({"topic":"Rohit Sharma"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Rohit Sharma'}`


Page: Rohit Sharma
Summary: Rohit Sharma (born 30 April 1987) is an Indian international cricketer and the captain of Indian cricket team in Test and ODI formats. He is widely regarded as one of the gHere's a 3-paragraph article on Rohit Sharma:

**The King of Cricket: Rohit Sharma's Unmatched Success**

Rohit Sharma, born on April 30, 1987, is an Indian international cricketer and the captain of the Indian cricket team in Test and ODI formats. He is widely regarded as one of the greatest batsmen in the history of cricket. With his incredible skills and unmatched success, Rohit has become a household name in India and around the world.

Rohit's journey to becoming a cricketer was not an easy one. He started playing cricket at a young age and quickly rose through the ranks. He made his debut for Mumbai in 2008 and went on to play for the Indian national team in 2010. Since then, he has been

In [50]:
display(Markdown(response['output']))

Here's a 3-paragraph article on Rohit Sharma:

**The King of Cricket: Rohit Sharma's Unmatched Success**

Rohit Sharma, born on April 30, 1987, is an Indian international cricketer and the captain of the Indian cricket team in Test and ODI formats. He is widely regarded as one of the greatest batsmen in the history of cricket. With his incredible skills and unmatched success, Rohit has become a household name in India and around the world.

Rohit's journey to becoming a cricketer was not an easy one. He started playing cricket at a young age and quickly rose through the ranks. He made his debut for Mumbai in 2008 and went on to play for the Indian national team in 2010. Since then, he has been a regular member of the team and has played in numerous international tournaments, including the ICC World Cup and the Champions Trophy.

Throughout his career, Rohit has achieved numerous milestones and broken records. He is the highest run-scorer in the history of the Indian Premier League (IPL) and has also become the fastest batsman to reach 10,000 runs in international cricket. With his incredible talent and dedication, Rohit Sharma continues to be a force to be reckoned with in the world of cricket.